# Retrieval 3: Multi-temporal parameters

This example shows how to use the [rt1 python package](https://github.com/raphaelquast/rt1_model) together with [scipy optimize](https://docs.scipy.org/doc/scipy/reference/optimize.html) to setup a retrieval procedure to 

- obtain multiple dynamic parameters from a series of incidence-angle dependent $\sigma^0$ measurements.

> **NOTE:** The plots in this example require the additional dependencies [matplotlib](https://matplotlib.org/) and [ipympl](https://matplotlib.org/ipympl/).


In [1]:
%matplotlib widget
from rt1_model import RT1, surface, volume, set_loglevel
from scipy.optimize import least_squares
import numpy as np

set_loglevel("info")

## Specify simulation and fit parameters
### Set parameter values that are used to simulate the data

In [2]:
dB, sig0 = False, True

num = 100  # Number of measurements
incs = 30  # Available incidence angles per measurement
noise_sigma = 0.5 if dB is True else 1e-3  # Noise-level (sigma of gaussian noise)

inc = np.random.normal(45, 10, (num, incs)).clip(20, 70)        # Incidence angles
N = np.random.normal(0.1, 0.1, (num, 1)).clip(0.01, 0.25)       # NormBRDF values
tau = (np.sin(np.linspace(0, 2*np.pi, num))**2)[:,np.newaxis]   # Optical Depth values

sim_params = dict(tau=tau, omega=0.2, N=N)  # Simulation parameter values
const_params = dict(t_s=0.4)                # Constant parameters (assumed to be known)

### Set start values and boundaries for the fit

In [3]:
start_vals = dict(omega=[0.2], tau=[0.3] * num, N=[0.1] * num)
bnd_vals = dict(omega=[(0.01, 0.5)], tau=[(0.01, 1.)] * num, N=[(0.01, 0.5)] * num)

## Setup RT1 and create a simulated dataset

In [4]:
V = volume.Rayleigh()
SRF = surface.HG_nadirnorm(t="t_s", ncoefs=10)

R = RT1(V=V, SRF=SRF, int_Q=True, dB=dB, sig0=sig0)
R.set_monostatic(p_0=0)
R.NormBRDF = "N"  # Use a synonym for NormBRDF parameter

R.set_geometry(t_0=np.deg2rad(inc))
tot = R.calc(**sim_params, **const_params)[0]

# Add some random noise
tot += np.random.normal(0, noise_sigma, tot.shape)

12:54:18.781 INFO: Evaluating coefficients for interaction-term...
12:54:18.897 INFO: Coefficients extracted, it took 0.01349 sec.
D:\python_modules\RT1_new\src\rt1_model\_calc.py:1013: RuntimeWarning: invalid value encountered in add
  np.exp(-tau / mu) * np.log(mu / (1.0 - mu))


D:\python_modules\RT1_new\src\rt1_model\_calc.py:1194: RuntimeWarning: invalid value encountered in add
  np.exp(-tau / mu) * np.log(mu / (1.0 - mu))


## Setup scipy optimize to fit RT1 model to the data

In [5]:
from scipy.sparse import csr_matrix, vstack, block_diag

def parse_params(x):
    """Map 1D parameter array to dict {parameter_name: value(s)}."""
    return dict(omega=x[0], tau=x[1:num+1][:, np.newaxis], N=x[num+1:][:, np.newaxis])

def fun(x):
    """Calculate residuals."""
    R.update_params(**parse_params(x), **const_params)
    res = (R.calc()[0] - tot).ravel() # Ravel result because scipy requires 1D arrays
    return res

def jac(x):
    """Calculate jacobian."""
    R.update_params(**parse_params(x), **const_params)
    jac = R.jacobian(param_list=["omega", "tau", "N"], format="scipy_least_squares")
    return jac


# Unpack start-values and boundaries as required by scipy optimize
x0 = [*start_vals["omega"], *start_vals["tau"], *start_vals["N"]]
bounds = list(zip(*[*bnd_vals["omega"], *bnd_vals["tau"], *bnd_vals["N"]]))

res = least_squares(
    fun=fun,
    x0=x0,
    bounds=bounds,
    jac=jac,
    ftol=1e-5,
    gtol=1e-5,
    xtol=1e-5,
    verbose=2,
)

# Unpack found parameters
found_params = parse_params(res.x)
# Calcuate total backscatter based on found parameters
found_tot = R.calc(**found_params, **const_params)[0]

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         3.2503e+00                                    1.07e+00    
       1              2         6.6168e-01      2.59e+00       1.44e+00       5.37e-01    
       2              3         2.2128e-01      4.40e-01       2.64e+00       1.88e+00    
       3              4         4.9313e-02      1.72e-01       9.90e-01       4.40e-01    
       4              5         2.4015e-02      2.53e-02       7.99e-01       3.52e-01    
       5              6         8.5223e-03      1.55e-02       5.27e-01       2.48e-01    
       6              7         3.9352e-03      4.59e-03       2.54e-01       7.97e-02    
       7              8         2.8764e-03      1.06e-03       2.17e-01       5.22e-02    
       8              9         2.4735e-03      4.03e-04       6.06e-02       3.56e-02    
       9             10         2.1072e-03      3.66e-04       7.40e-02       2.29e-02    

       8              9         2.3216e-02      1.09e-02       1.82e-01       2.40e-01    
       9             10         7.5239e-03      1.57e-02       4.81e-01       1.89e-01    
      10             11         4.7438e-03      2.78e-03       1.67e-01       9.31e-02    
      11             12         4.0919e-03      6.52e-04       7.60e-02       6.34e-02    


      12             13         3.7649e-03      3.27e-04       3.26e-02       4.97e-02    
      13             14         3.1768e-03      5.88e-04       6.28e-02       2.63e-02    
      14             15         2.7132e-03      4.64e-04       8.43e-02       1.54e-02    
      15             16         2.4762e-03      2.37e-04       2.22e-02       8.79e-03    


      16             17         2.3107e-03      1.66e-04       1.78e-02       8.21e-03    
      17             18         2.1243e-03      1.86e-04       1.75e-02       7.38e-03    
      18             19         1.9897e-03      1.35e-04       1.58e-02       6.39e-03    
      19             20         1.9177e-03      7.20e-05       1.09e-02       5.66e-03    


      20             21         1.8686e-03      4.91e-05       8.88e-03       5.05e-03    
      21             22         1.8311e-03      3.75e-05       7.78e-03       4.48e-03    
      22             23         1.8037e-03      2.75e-05       6.25e-03       3.94e-03    
      23             24         1.7916e-03      1.21e-05       4.20e-03       3.69e-03    


      24             25         1.7780e-03      1.36e-05       4.05e-03       3.44e-03    
      25             26         1.7669e-03      1.12e-05       3.40e-03       3.22e-03    
      26             27         1.7507e-03      1.61e-05       4.46e-03       2.86e-03    
      27             28         1.7429e-03      7.79e-06       3.13e-03       2.65e-03    


      28             29         1.7378e-03      5.18e-06       2.47e-03       2.50e-03    
      29             30         1.7310e-03      6.77e-06       2.22e-03       2.28e-03    
      30             31         1.7250e-03      6.01e-06       1.85e-03       2.07e-03    
      31             32         1.7202e-03      4.76e-06       1.65e-03       1.88e-03    


      32             33         1.7153e-03      4.96e-06       1.34e-03       1.65e-03    
      33             34         1.7094e-03      5.90e-06       1.82e-03       1.35e-03    
      34             35         1.7056e-03      3.80e-06       1.29e-03       1.11e-03    
      35             36         1.7037e-03      1.89e-06       1.09e-03       9.78e-04    


      36             37         1.7014e-03      2.30e-06       1.16e-03       7.89e-04    
      37             38         1.7012e-03      2.01e-07       1.29e-04       5.52e-04    
      38             39         1.6979e-03      3.29e-06       1.38e-03       8.68e-04    
      39             40         1.6973e-03      6.26e-07       6.58e-04       3.09e-04    
      40             41         1.6968e-03      4.88e-07       5.53e-04       2.00e-04    


      41             42         1.6966e-03      1.68e-07       5.02e-04       1.39e-04    
      42             43         1.6965e-03      1.04e-07       1.06e-03       1.04e-04    
      43             44         1.6964e-03      9.70e-08       1.94e-03       1.49e-04    
      44             45         1.6963e-03      9.57e-08       1.62e-03       1.81e-04    
      45             46         1.6963e-03      4.10e-08       1.30e-03       2.36e-04    


      46             47         1.6962e-03      2.45e-08       7.41e-04       2.49e-04    
      47             48         1.6962e-03      2.31e-08       8.08e-04       2.73e-04    
      48             49         1.6962e-03      2.20e-08       8.63e-04       3.01e-04    
      49             50         1.6962e-03      1.87e-08       8.09e-04       3.32e-04    
      50             51         1.6962e-03      1.59e-08       7.29e-04       3.64e-04    
`ftol` termination condition is satisfied.
Function evaluations 51, initial cost 3.3004e+00, final cost 1.6962e-03, first-order optimality 3.64e-04.


## Visualize Results
### Plot timeseries

In [6]:
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter

f, (ax, ax2) = plt.subplots(2, figsize=(10, 4), sharex=True)

# Plot retrieved parameter timeseries
ax.set_ylabel("N / tau")

ax.plot(sim_params["N"], marker=".", lw=0.25, label="target N", c="C0")
ax.plot(found_params["N"], marker="o", lw=0.25, markerfacecolor="none", label="retrieved N", c="C0")

ax.plot(sim_params["tau"], marker=".", lw=0.25, label="target tau", c="C1")
ax.plot(found_params["tau"], marker="o", lw=0.25, markerfacecolor="none", label="retrieved tau", c="C1")
ax.plot(savgol_filter(found_params["tau"].ravel(), 15, 3), lw=1, c="C1", ls="--", label="retrieved tau (smoothed)")

# Plot backscatter timeseries
ax2.set_ylabel(r"$\sigma_0$ [dB]")
ax2.set_xlabel("# measurement")

ax2.plot(tot, lw=0, marker=".", c="C0", ms=3)
ax2.plot(found_tot, lw=0, marker="o", markerfacecolor="none", c="C1", ms=3)

ax.legend(loc="upper center", ncols=3, bbox_to_anchor=(0.5, 1.5))
f.tight_layout()

### Initialize analyzer widget and overlay results

12:54:29.524 INFO: Evaluating coefficients for interaction-term...
12:54:29.579 INFO: Coefficients extracted, it took 0.01470 sec.


In [7]:
analyze_params = {key: (*np.mean(np.atleast_2d(bnd_vals[key]), axis=0), found_params[key].mean()) for key in found_params}

ana = R.analyze(**analyze_params)

# Plot fit-data on top
ana.ax.scatter(inc, tot, c="k", s=3, zorder=0)
ana.ax.scatter(inc, found_tot, c="C0", s=1, zorder=0)

# Indicate fit-results in slider-axes
for key, s in ana.sliders.items():
    if key in ["omega"]:
        s.ax.plot(sim_params[key], np.mean(s.ax.get_ylim()), marker="o")

# Add text for static parameters
t = ana.f.text(
    0.6,
    0.95,
    "\n".join(
        [
            f"{key:>8} = {found_params[key]:.3f} ({sim_params[key]:.2f})  "
            rf"| $\Delta$ = {found_params[key] - sim_params[key]: .3f}"
            for key in ["omega"]
        ]
    ),
    va="top",
    fontdict=dict(family="monospace", size=8),
)